# L03.1 Interactive plots with Bokeh

In order for this lecture to be standalone, we will use the dataset by the [Gapminder foundation](https://www.gapminder.org/). This dataset is very rich and allows us to demonstrate a rich demonstration of Bokeh's capabilities.

In [3]:
from gapminder import gapminder
df = gapminder

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 6 columns):
country      1704 non-null object
continent    1704 non-null object
year         1704 non-null int64
lifeExp      1704 non-null float64
pop          1704 non-null int64
gdpPercap    1704 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 80.0+ KB


In [5]:
df['country'].unique() # Display unique entries in row 'country'

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Australia', 'Austria', 'Bahrain', 'Bangladesh', 'Belgium',
       'Benin', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Ethiopia',
       'Finland', 'France', 'Gabon', 'Gambia', 'Germany', 'Ghana',
       'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Haiti',
       'Honduras', 'Hong Kong, China', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kenya', 'Korea, Dem. Rep.',
       'Korea, Rep.', 'Kuwait', 'Leba

In [6]:
df.head(10) # display first 10 entries

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
5,Afghanistan,Asia,1977,38.438,14880372,786.113360
6,Afghanistan,Asia,1982,39.854,12881816,978.011439
7,Afghanistan,Asia,1987,40.822,13867957,852.395945
8,Afghanistan,Asia,1992,41.674,16317921,649.341395
9,Afghanistan,Asia,1997,41.763,22227415,635.341351


Looking at the head of the dataframe tells us that we have information on life expectancies, population, and GDP for each country every 5 years. This should allow us to produce some interesting plots!

## Bokeh

### Line charts

We already used line charts in the basic plotting example. Let's check out how life expectancy has been doing over the years in Europe.

In [7]:
is_europe = df['continent'] == "Europe"
europe = df[is_europe]
europe.head()

,country,continent,year,lifeExp,pop,gdpPercap
12,Albania,Europe,1952,55.23,1282697,1601.056136
13,Albania,Europe,1957,59.28,1476505,1942.284244
14,Albania,Europe,1962,64.82,1728137,2312.888958
15,Albania,Europe,1967,66.22,1984060,2760.196931
16,Albania,Europe,1972,67.69,2263554,3313.422188


In [85]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Bokeh does not automatically cycle through colours
from bokeh.palettes import Category20_20 as palette
import itertools
colors = itertools.cycle(palette) 

output_notebook()

countries = europe['country'].unique()

# Create a blank figure with labels
fig = figure(plot_width = 600, plot_height = 600, 
           title = 'Gapminder data',
           x_axis_label = 'year', y_axis_label = 'life_expectancy')

# Creating traces by looping over countries
for country in countries:
    is_country = europe['country'] == country
    fig.line(europe[is_country]['year'],
             europe[is_country]['lifeExp'],
             line_width=2,
             color=next(colors))
    
show(fig)

Loading BokehJS ...

In [18]:
colors = itertools.cycle(palette) 

scandinavians = ['Norway', 'Sweden', 'Denmark']
is_scandinavia = europe['country'].isin(scandinavians)
scandinavia = europe[is_scandinavia]
 
fig = figure(plot_width = 600, plot_height = 600, 
           title = 'Gapminder data',
           x_axis_label = 'year', y_axis_label = 'life_expectancy')

for country in scandinavians:
    is_country = europe['country'] == country
    fig.line(europe[is_country]['year'],
             europe[is_country]['lifeExp'],
             line_width=2,
             color=next(colors))
    
show(fig)

### Bar charts

To illustrate bar charts we'll look at the GDP in Scandinavia over the years.

In [83]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Category20_20 as palette
from bokeh.transform import factor_cmap

years = [str(year) for year in list(europe['year'].unique())]
data = scandinavia[['country', 'year', 'gdpPercap']]

data = scandinavia.to_dict(orient='list')
x = [(year, country) for country in scandinavians for year in years]
data['x'] = x

source = ColumnDataSource(data=data)

fig = figure(x_range=FactorRange(*x), plot_height=250, title="Gapminder data",
           toolbar_location=None, tools="")
fig.vbar(x='x', top='gdpPercap', width=0.8, source=source,
        fill_color=factor_cmap('x', palette=palette, factors=scandinavians, start=1, end=2))

fig.xaxis.major_label_orientation = 1
    
show(fig)

### Bubble plots

To illustrate scatter plots we'll look at life expectancy vs. income per country, where we'll scale the bubble by population size. This is the most well known graphic by Gapminder.

In [ ]:
import numpy as np

fig = go.Figure()

is_year2002 = df['year'] == 2002
year2002 = df[is_year2002]

continents = year2002['continent'].unique()

# set hover text
hover_text = []
for index, row in year2002.iterrows():
    hover_text.append(('Country: {country}<br>'+
                      'Life Expectancy: {lifeExp}<br>'+
                      'GDP per capita: {gdp}<br>'+
                      'Population: {pop}<br>'+
                      'Year: {year}').format(country=row['country'],
                                            lifeExp=row['lifeExp'],
                                            gdp=row['gdpPercap'],
                                            pop=row['pop'],
                                            year=row['year']))
year2002['text'] = hover_text

# loop over continents
for continent in continents:
    is_continent = year2002['continent'] == continent
    pop = np.array(year2002[is_continent]['pop'].values, dtype=int)
    sizeref = 2.*max(pop)/(100)
    
    countries = year2002[is_continent]['country'].values

    trace = go.Scatter(x=year2002[is_continent]['gdpPercap'],
                    y=year2002[is_continent]['lifeExp'],
                    name=continent,
                    marker_size=pop,
                    text=year2002[is_continent]['text'])
    fig.add_trace(trace)
    fig.update_traces(mode='markers', marker=dict(sizemode='area',
                                                  sizeref=sizeref, line_width=2))

fig.update_layout(xaxis=dict(
        title="GDP per capita",
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
    ),
    yaxis=dict(
        title="life expectancy",
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
    ),
    autosize=False,
)

fig.show()

This is a nice plot, but it's only one year. How did this relationship evolve over the years? To do this we use the ```FigureWidget``` object that we can pass a ```Figure``` object.

In [ ]:
widget = go.FigureWidget(fig)
widget.show()

Adding interactivity to a widget requires some extra imports:

In [ ]:
from ipywidgets import widgets
from IPython.display import display, clear_output, Image
from chart_studio.widgets import GraphWidget